In [13]:
from extract_code import *

In [14]:
import os
current_path = os.getcwd()

In [26]:
prueba_json = get_project_structure(current_path)

In [52]:
extract_code_snippets(prueba_json)

['    def __init__(self, model_path, code_snippets, device=None):\n        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else device\n        self.tokenizer = RobertaTokenizer.from_pretrained(\'microsoft/unixcoder-',
 "base')\n        self.model = self.load_model(model_path)\n        self.code_snippets = code_snippets\n        self.code_embeddings = self.generate_code_embeddings(code_snippets)",
 '    def forward(self, code_inputs=None, nl_inputs=None):\n        if code_inputs is not None:\n            outputs = self.encoder(code_inputs,attention_mask=code_inputs.ne(1))[0]\n            outputs = (outputs*code_inputs.ne(1)[:,:,None]).sum(1)/code_inputs.',
 'ne(1).sum(-1)[:,None]\n            return torch.nn.functional.normalize(outputs, p=2, dim=1)\n        else:\n            outputs = self.encoder(nl_inputs,attention_mask=nl_inputs.ne(1))[0]\n            outputs = (outputs*nl_inputs.ne(1)[:,:,None]).sum(1)/nl_in',
 'puts.ne(1).sum(-1)[:,No

In [20]:
import argparse
import os
import json
import subprocess
import tempfile
import shutil


def clone_github_repo(github_url, dest_folder):
    """
    Clone a GitHub repository to the specified local directory, even if the directory is not empty.
    """
    try:
        # Create a temporary directory to clone the repo
        with tempfile.TemporaryDirectory() as temp_dir:
            # Clone the repository into the temporary directory
            subprocess.run(["git", "clone", github_url, temp_dir], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
            # Move the contents of the cloned repo to the destination folder
            repo_name = os.path.basename(github_url.rstrip('/').split('/')[-1])
            source_folder = os.path.join(temp_dir, repo_name)
            for item in os.listdir(source_folder):
                s = os.path.join(source_folder, item)
                d = os.path.join(dest_folder, item)
                if os.path.isdir(s):
                    shutil.copytree(s, d, dirs_exist_ok=True)
                else:
                    shutil.copy2(s, d)
            
            print("Repository cloned successfully.")

    except subprocess.CalledProcessError as e:
        print(f"Error in subprocess: {e.stderr.decode().strip()}")
        raise
    except Exception as e:
        print(f"Error: {e}")
        raise



In [17]:
from code_searcher import CodeSearcher, Model

In [33]:
from urllib.parse import urlsplit

In [46]:
def extract_code_snippets(json_data, max_length=256):
    code_snippets = []

    def add_snippet(code):
        for i in range(0, len(code), max_length):
            code_snippets.append(code[i:i + max_length])

    def process_dict(a_dict):
        for key, value in a_dict.items():
            if isinstance(value, dict):
                if 'code' in value:
                    add_snippet(value['code'])
                else:
                    process_dict(value)

    process_dict(json_data)
    return code_snippets



In [35]:
from urllib.parse import urlsplit

def get_last_segment(url):
    """
    Extracts the last part (segment) of a URL's path.

    Args:
    url (str): The URL to extract the last segment from.

    Returns:
    str: The last segment of the URL's path.
    """
    parsed_url = urlsplit(url)
    path_segments = parsed_url.path.split('/')
    last_segment = path_segments[-1]
    return last_segment

# Example usage:
url = "https://www.example.com/path/to/resource"
last_segment = get_last_segment(url)
print(last_segment)  # Output: "resource"



resource


In [55]:

github_url = "https://github.com/dr-aheydari/SoftAdapt"
output_dir_path = r"C:\Users\agianolini\OneDrive - ANDES WEALTH MANAGEMENT SA\Desktop\research-assistant-main"
model_path = r"C:\Users\agianolini\OneDrive - ANDES WEALTH MANAGEMENT SA\Desktop\research-assistant-main\model (7).bin"
nl_query = 'importing softadapt'



temp_dir = r"C:\Users\agianolini\OneDrive - ANDES WEALTH MANAGEMENT SA\Desktop\research-assistant-main\output" # Temporary directory to clone the repo
clone_github_repo(github_url, temp_dir)
repo_name = get_last_segment(github_url)

# Get the project structure
project_structure = get_project_structure(temp_dir )

# Save the project structure to a JSON file
os.makedirs(output_dir_path, exist_ok=True)
project_name = os.path.basename(github_url)
output_path = os.path.join(output_dir_path, f'{project_name}.json')


with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(project_structure, f, ensure_ascii=False, indent=4)

data = {}
# Leer y analizar el archivo JSON
try:
    with open(output_path, 'r') as file:
        data = json.load(file)
except Exception as e:
    print(f"Error al leer el archivo JSON: {e}")

# Proceso de extracción de fragmentos de código
code_snippets = []
code_snippets = extract_code_snippets(data)
# try:
# # Iterar sobre todas las claves principales del diccionario JSON
#     for main_key in data.keys():
#         for filename, content in data[main_key].items():
#             if 'functions' in content:
#                 for function_name, function_content in content['functions'].items():
#                     if 'code' in function_content:
#                         code_snippets.append(function_content['code'])
# except Exception as e:
#     code_snippets = f"Error al procesar el JSON: {e}"

searcher = CodeSearcher(model_path, code_snippets)
k=3
t = searcher.get_similarity_search(nl_query, k)

print("Top K similar items: \n")
print(t)

# print(data)
# print(code_snippets)


Repository cloned successfully.
Top K similar items: 

{'top_3': {'index': 34, 'snippet': 'class NormalizedSoftAdapt(SoftAdaptBase):\n    """The normalized-slopes variant class.\n\n    The normalized variant of SoftAdapt is described in section 3.1.3 of our\n    manuscript (located at: https://arxiv.org/pdf/1912.12355.pdf).\n\n    Attributes:\n        ', 'similarity': 0.6222495311109397}, 'top_2': {'index': 55, 'snippet': 'class SoftAdapt(SoftAdaptBase):\n    """The original variant class.\n\n    The original variant of SoftAdapt is described in section 3.1.1 of our\n    manuscript (located at: https://arxiv.org/pdf/1912.12355.pdf).\n\n    Attributes:\n        beta: A float that is', 'similarity': 0.6070806460289828}, 'top_1': {'index': 11, 'snippet': 'class LossWeightedSoftAdapt(SoftAdaptBase):\n    """Class implementation of the loss-weighted SoftAdapt variant.\n\n    The loss-weighted variant of SoftAdapt is described in section 3.1.1 of our\n    manuscript (located at: https://arxi

In [56]:
t

{'top_3': {'index': 34,
  'snippet': 'class NormalizedSoftAdapt(SoftAdaptBase):\n    """The normalized-slopes variant class.\n\n    The normalized variant of SoftAdapt is described in section 3.1.3 of our\n    manuscript (located at: https://arxiv.org/pdf/1912.12355.pdf).\n\n    Attributes:\n        ',
  'similarity': 0.6222495311109397},
 'top_2': {'index': 55,
  'snippet': 'class SoftAdapt(SoftAdaptBase):\n    """The original variant class.\n\n    The original variant of SoftAdapt is described in section 3.1.1 of our\n    manuscript (located at: https://arxiv.org/pdf/1912.12355.pdf).\n\n    Attributes:\n        beta: A float that is',
  'similarity': 0.6070806460289828},
 'top_1': {'index': 11,
  'snippet': 'class LossWeightedSoftAdapt(SoftAdaptBase):\n    """Class implementation of the loss-weighted SoftAdapt variant.\n\n    The loss-weighted variant of SoftAdapt is described in section 3.1.1 of our\n    manuscript (located at: https://arxiv.org/pdf/1912.12355.pd',
  'similarity': 0